# Bitta Parking
Il Bitta Parking è un parcheggio a 5 livelli con numero variabile di posti, riservato esclusivamente agli Alfisti

In [67]:
# per importare la "cosa" che permette di leggere la data e l'ora a python

from datetime import datetime

In [68]:
parking = {"level1": {"total_parking_spaces": 10,
                      "taken_parking_spaces": 5},
            "level2": {"total_parking_spaces": 20,
                      "taken_parking_spaces": 7},
            "level3": {"total_parking_spaces": 39,
                      "taken_parking_spaces": 38},
            "level4": {"total_parking_spaces": 50,
                      "taken_parking_spaces": 50},
            "level5": {"total_parking_spaces": 5,
                      "taken_parking_spaces": 0}
            }

In [69]:
car = {"AX555LM": {"entry": datetime.now(), 
                   "level": "level1"},
        "CJ687LI": {"entry": datetime.now(), 
                   "level": "level3"}
        }


## Requisiti:
- occupiamo prima i piani inferiori;
- l'ultimo piano è riservato alla famiglia Bitta;
- i clienti pagano in base alla permanenza una quota proporzionata(€50/h);
- sono accettati esclusivamente possessori di Alfa Romeo e Camper.

In [70]:
# check if the vehicle is either an Alfa Romeo or a Camper 

def check_vehicle():
    model = input("Is this vehicle an Alfa Romeo or a Camper?  [Y/N]").lower()
    if model == "y":
        return True
    elif model == "n":
        return False
    else:
        check_vehicle() #in questo else, se la risposta non è né sì né n c'è una funzione ricorsiva, quinsi si ripropone la domanda fino a che la risposta non è o y o n
        

In [71]:
# check if the driver is a member of the Bitta Family

def is_Bitta_member():
    member = input("Are you a Bitta? [Y/N]").lower()
    if member == "y":
        return True
    elif member == "n":
        return False 
    else:
        is_Bitta_member()

In [72]:
def is_standard_spot_avaialble():
    for level, level_info in parking.items():
        if level == "level5":
            break
        total_s_parking_spaces = level_info["total_parking_spaces"]
        taken_s_parking_spaces = level_info["taken_parking_spaces"]
        Sspot_available = (total_s_parking_spaces - taken_s_parking_spaces) > 0

        if Sspot_available:
            return Sspot_available, level
        
    return Sspot_available, level # l'ultimo check che va a  fare se non ci sono posti disponibili in nessun livello, esce dal ciclo e ritorna false perché il quinto non viene considerato


In [73]:
# check if there's any avaialble parking space for the Bittas

def is_Bitta_spot_available():
    total_B_parking_spaces = parking["level5"]["total_parking_spaces"]
    taken_B_parking_spaces = parking["level5"]["taken_parking_spaces"]
    Bspot_available = (total_B_parking_spaces - taken_B_parking_spaces) > 0 
    return Bspot_available


In [74]:
# questa funzione ha il ruolo di aggiungere +1 nei posti presi nel dict parking usando la 
# chiave level (quindi nel livello specifico), concatenandosi con le funzioni di prima

def add_taken_spot(level):
    parking[level]["taken_parking_spaces"] += 1
    

In [75]:
def add_car_to(level):
    plate = input("Insert your license plate: ").upper()
    car[plate] = {} #aggiungiamo la targa all'auto che sta entrando nel parcheggio
    car[plate]["entry"] = datetime.now() # aggiungiamo l'orario di accesso della data macchina
    car[plate]["level"] = level # aggiungiamo il livello in cui l'auto deve andare
    add_taken_spot(level) # va ad aggiungere un posto preso
    

In [76]:
def add_car(level):
    add_car_to(level)
    add_taken_spot(level)

In [77]:
def get_car_info(plate):
    plate_dict = car[plate] # seleziona le info sulle targhe
    entry_time = plate_dict["entry"] # prende l'orario di entrate
    level = plate_dict["level"] # prende il livello dove l'auto si trova 
    return level, entry_time

In [78]:
# questa funzione, contestualmente alla funzione add_taken_spot, ha il ruolo di togliere -1 auto al 
# livello specifico, concatenandosi alle funzioni precedenti

def remove_taken_spot(level):
    parking[level]["taken_parking_spaces"] -= 1
    

In [79]:
# to remove the car (and its plate) from the dictionary

def remove_plate(plate):
    del car[plate]
    

In [80]:
def remove_car(plate, level):
    remove_plate(plate)
    remove_taken_spot(level)

In [81]:
# per calcolare le ore totali di permanenza nel parking

def calculate_hours(entry_time):
    diff = datetime.now() - entry_time
    hours = diff.total_seconds() // 3600
    return hours + 1


In [82]:
def calculate_fare(entry_time):
    hours = calculate_hours(entry_time)
    hour_fare = 50
    fare = hour_fare * hours
    return fare

In [83]:
def car_in():
    is_Alfa = check_vehicle()
    if not is_Alfa:
        print("You shall not pass!")
        return 
    is_Bitta = is_Bitta_member()

    # se è un Bitta
    if is_Bitta:
        spot_available = is_Bitta_spot_available()
        if not spot_available:
            print("There are too many Bittas on the 5th level, we're sorry there's no spot left.")
            return # qui se non c'è posto non si fa nulla
        else:
            selected_level = "level5"
    
    # se non è un Bitta
    else:
        spot_available, level = is_standard_spot_avaialble()
        if not spot_available:
            print("We are sorry there are no spots available in the parking.")
            return
        else: 
            selected_level = level 

    # se c'è posto per i non-Bitta aggiungiamo la targa
    add_car(selected_level)


In [84]:
car_in()

In [85]:
car

{'AX555LM': {'entry': datetime.datetime(2025, 1, 30, 12, 57, 29, 312926),
  'level': 'level1'},
 'CJ687LI': {'entry': datetime.datetime(2025, 1, 30, 12, 57, 29, 312926),
  'level': 'level3'},
 'DF538UH': {'entry': datetime.datetime(2025, 1, 30, 12, 57, 41, 406915),
  'level': 'level1'}}

In [90]:
def car_out():

    # check della targa nel parcheggio
    plate = input("Insert your license plate please:").upper()
    if not plate in car:
        print("There's no car with this license plate in the parking!")
        return
    
    # ora andiamo a "raggruppare" tutte le funzioni che prima abbiamo definito per rimuovere l'auto, di modo da
    # rimuoverla dal dict e calcolare le ore e la tariffa che deve pagare
    level, entry_time = get_car_info(plate)
    remove_car(plate, level)

    fare = calculate_fare(entry_time)
    print(f"You have to pay €{fare}. Thanks!")


In [92]:
car_out()

You have to pay €50.0. Thanks!
